<a href="https://colab.research.google.com/github/vaishnavipatil29/Emotion-Detection-/blob/main/CNN_Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNN**

Why cnn?
CNNs are very effective in reducing the number of parameters without losing on the quality of models. Images have high dimensionality (as each pixel is considered as a feature) which suits the above described abilities of CNNs. CNN's are really effective for image classification as the concept of dimensionality reduction suits the huge number of parameters in an image

In [ ]:
import numpy as np
import pandas as pd

import keras as k
data=pd.read_csv('../input/fer2013/fer2013.csv')

Using TensorFlow backend.


In [ ]:
train_set = data[(data.Usage == 'Training')] 
val_set = data[(data.Usage == 'PublicTest')]
test_set = data[(data.Usage == 'PrivateTest')] 

In [ ]:
y_train=train_set['emotion']

In [ ]:
X_train = np.array(list(map(str.split, train_set.pixels)), np.float32)

X_val = np.array(list(map(str.split, val_set.pixels)), np.float32)
X_test = np.array(list(map(str.split, test_set.pixels)), np.float32) 
Y_val=val_set['emotion']
y_test=test_set['emotion']


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1) 
X_val = X_val.reshape(X_val.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
from keras.utils import np_utils,to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


In [ ]:
len(y_train)

28709

In [ ]:
y_train = to_categorical(y_train, 7)
Y_val = to_categorical(Y_val, 7)


In [ ]:
#CNN
cnn4 = Sequential() #Sequential model allows to build a model layer by layer

#--------------------------------Input layer------------------------------------------# 
#COnvolution layer parametrs:
#No of output filters: 32
#kernel_size: specifying the height and width of the 2D convolution window : (3,3)
#activation function : relu
#input_shape : shape of image matrix : 48x48x1 : (Gray Scale Image)
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1))) 
#Batch Normalization : It is used to normalize the output of the previous layers. The activations scale the input layer in normalization.
#Using batch normalization learning becomes efficient also it can be used as regularization to avoid overfitting of the model.  
cnn4.add(BatchNormalization())

#--------------------------------Second layer------------------------------------------# 
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu')) 
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2))) #Down-sampling opertaion to reduce dimensionality of feature map
cnn4.add(Dropout(0.25)) #regularization technique to reduce overfitting: dropouts randomly switches off some neurons in the network which forces the data to find new paths

#--------------------------------Third layer------------------------------------------# 
cnn4.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

#--------------------------------Fourth layer------------------------------------------# 
cnn4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Flatten()) #converts 3d vector to 1 d vector : Flatten serves as a connection between the convolution and dense layers.

#--------------------------------Output layers------------------------------------------# 
#‘Dense’ is the layer type we will use in for our output layer. 
cnn4.add(Dense(128, activation='relu')) #Dense layer predict labels
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(128, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(7, activation='softmax'))        #FINAL PREDICTION
#Softmax makes the output sum up to 1 so the output can be interpreted as probabilities.

In [ ]:
#Parameters:
#1) The optimizer controls the learning rate. Optmizerused : 'adam'. The adam optimizer adjusts the learning rate throughout training.
#2) Loss function used: ‘categorical_crossentropy". A lower score indicates that the model is performing better.
#3) To make things even easier to interpret, ‘accuracy’ metric is used to see the accuracy score on the validation set when we train the model.
cnn4.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
#The batch size defines the number of samples that will be propagated through the network.(at a time)
cnn4.fit(X_train,
         y_train,
         batch_size=120,
         epochs=30,
         validation_data=(X_val, Y_val)
         )

Train on 28709 samples, validate on 3589 samples
Epoch 1/30
28709/28709 [==============================] - 13s 451us/step - loss: 2.1235 - accuracy: 0.2791 - val_loss: 1.5957 - val_accuracy: 0.3884
Epoch 2/30
28709/28709 [==============================] - 8s 281us/step - loss: 1.6200 - accuracy: 0.3925 - val_loss: 1.4164 - val_accuracy: 0.4533
Epoch 3/30
28709/28709 [==============================] - 8s 277us/step - loss: 1.4369 - accuracy: 0.4496 - val_loss: 1.4469 - val_accuracy: 0.4294
Epoch 4/30
28709/28709 [==============================] - 8s 281us/step - loss: 1.3534 - accuracy: 0.4808 - val_loss: 1.2959 - val_accuracy: 0.5130
Epoch 5/30
28709/28709 [==============================] - 8s 287us/step - loss: 1.3009 - accuracy: 0.5044 - val_loss: 1.2449 - val_accuracy: 0.5366
Epoch 6/30
28709/28709 [==============================] - 8s 283us/step - loss: 1.2498 - accuracy: 0.5257 - val_loss: 1.2049 - val_accuracy: 0.5380
Epoch 7/30
28709/28709 [==============================] - 8s 2

Training accuracy : 75.45%

Validation accuracy : 61.33%

Thus, CNN gives higher accuracy than ML algorithms.